In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="qwen-max-latest", temperature=0)

In [3]:
from langchain_core.prompts import PromptTemplate

# 基础少样本学习

In [4]:
def few_shot_sentiment_classification(input_text):
    few_shot_prompt = PromptTemplate(
        input_variables=["input_text"],
        template="""
        将情绪分类为正面、负面或中性。
        
        示例：
        文本：我喜欢这个产品！它太棒了。
        情绪：正面
        
        文本：这部电影很糟糕。我讨厌它。
        情绪：负面

        文本：今天的天气还可以。
        情绪：中性

        现在，对以下内容进行分类：
        文本：{input_text}
        情绪：
        """,
    )

    chain = few_shot_prompt | llm
    result = chain.invoke({"input_text": input_text}).content

    # Clean up the result
    result = result.strip()
    # Extract only the sentiment label
    if ":" in result:
        result = result.split(":")[1].strip()

    return result  # This will now return just "Positive", "Negative", or "Neutral"

In [5]:
test_text = "这家新餐厅这么棒！你敢信？"
result = few_shot_sentiment_classification(test_text)
print(f"输入: {test_text}")
print(f"预测情绪: {result}")

输入: 这家新餐厅这么棒！你敢信？
预测情绪: 正面


# 高级少样本技术

In [6]:
def multi_task_few_shot(input_text, task):
    few_shot_prompt = PromptTemplate(
        input_variables=["input_text", "task"],
        template="""
        对给定的文本执行指定的任务。

        示例：
        文本：我喜欢这个产品！它太棒了。
        任务：情绪
        结果：积极

        文本：你好，你好吗？
        任务：语言
        结果：英语

        现在，执行以下任务：
        文本：{input_text}
        任务：{task}
        结果：
        """,
    )

    chain = few_shot_prompt | llm
    return chain.invoke({"input_text": input_text, "task": task}).content

In [7]:
print(multi_task_few_shot("我真不敢相信这有多棒！", "情绪"))
print(multi_task_few_shot("I can't believe how great this is!", "语言"))

积极
英语


# 上下文学习

In [8]:
def in_context_learning(task_description, examples, input_text):
    example_text = "".join(
        [f"输入: {e['input']}\n输出: {e['output']}\n\n" for e in examples]
    )

    in_context_prompt = PromptTemplate(
        input_variables=["task_description", "examples", "input_text"],
        template="""
        任务: {task_description}
        
        示例:
        {examples}
        
        现在，根据以下输入执行任务：
        输入: {input_text}
        输出:
        """,
    )

    chain = in_context_prompt | llm
    return chain.invoke(
        {
            "task_description": task_description,
            "examples": example_text,
            "input_text": input_text,
        }
    ).content

In [12]:
task_desc = "将给定的文本翻译成猪拉丁。直接输出最终结果。"
examples = [
    {"input": "hello", "output": "ellohay"},
    {"input": "apple", "output": "appleay"},
]
test_input = "python"

result = in_context_learning(task_desc, examples, test_input)
print(f"输入: {test_input}")
print(f"输出: {result}")

输入: python
输出: ythonpay


# 最佳实践与评估

In [13]:
def evaluate_model(model_func, test_cases):
    """
    Evaluate the model on a set of test cases.

    Args:
    model_func: The function that makes predictions.
    test_cases: A list of dictionaries, where each dictionary contains an "input" text and a "label" for the input.

    Returns:
    The accuracy of the model on the test cases.
    """
    correct = 0
    total = len(test_cases)

    for case in test_cases:
        input_text = case["input"]
        true_label = case["label"]
        prediction = model_func(input_text).strip()

        is_correct = prediction.lower() == true_label.lower()
        correct += int(is_correct)

        print(f"输入: {input_text}")
        print(f"预测值: {prediction}")
        print(f"真实值: {true_label}")
        print(f"是否正确: {is_correct}\n")

    accuracy = correct / total
    return accuracy

In [15]:
test_cases = [
    {"input": "这个产品超出了我的预期！", "label": "正面"},
    {"input": "我对这项服务非常失望。", "label": "负面"},
    {"input": "今天最高气温23度", "label": "中性"},
]

accuracy = evaluate_model(few_shot_sentiment_classification, test_cases)
print(f"模型准确率: {accuracy:.2f}")

输入: 这个产品超出了我的预期！
预测值: 正面
真实值: 正面
是否正确: True

输入: 我对这项服务非常失望。
预测值: 负面
真实值: 负面
是否正确: True

输入: 今天最高气温23度
预测值: 中性
真实值: 中性
是否正确: True

模型准确率: 1.00
